LOAD MODELS AND DEPENDENCIES

In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# List files in your model directory
print(os.listdir('/content/drive/MyDrive/saved_models'))

['survey_model.h5', 'survey_scaler.pkl', 'survey_encoder.pkl', 'C6_values.npy', 'C5_values.npy', 'C4_values.npy', 'C3_values.npy', 'C2_values.npy', 'audio_model.h5', 'facial_model.h5']


In [3]:
#Loads pre-trained models from model directory
import tensorflow as tf
facial_model = tf.keras.models.load_model("/content/drive/MyDrive/saved_models/facial_model.h5") #Facial Prediction
audio_model = tf.keras.models.load_model("/content/drive/MyDrive/saved_models/audio_model.h5") #Audio Prediction

FACIAL

In [4]:
import cv2
import numpy as np
import tensorflow as tf

#Function that takes an image path and loads model
def predict_facial(photo_path, model):

    """
    photo_path: path to the photo file
    model: loaded keras facial model

    Returns:
        label: 'Stressed' or 'Not Stressed'
        confidence: float, confidence for the predicted label
    """

    # Load and preprocess image
    #Reads the image from photo path, converts it to grayscale, resized and normalizes it
    img = cv2.imread(photo_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    normalized = resized / 255.0
    input_img = normalized.reshape(1, 48, 48, 1)

    # Predict
    prediction = model.predict(input_img, verbose=0)
    class_id = np.argmax(prediction)
    confidence = float(prediction[0][class_id])
    label = "Stressed" if class_id == 1 else "Not Stressed"
    return label, confidence

SURVEY

In [5]:
import tensorflow as tf
import joblib
import numpy as np
import pandas as pd

# Load your trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/saved_models/survey_model.h5')

# Load the fitted encoder and scaler
encoder = joblib.load('/content/drive/MyDrive/saved_models/survey_encoder.pkl')
scaler = joblib.load('/content/drive/MyDrive/saved_models/survey_scaler.pkl')

In [6]:
# Load unique values for each question (for Gradio radio buttons)
features = ['C2', 'C3', 'C4', 'C5', 'C6']
choices = []
for col in features:
    choices.append(sorted(np.load(f"/content/drive/MyDrive/saved_models/{col}_values.npy", allow_pickle=True)))

#Function to predict stressed/not stressed based on survey answered by the user.
def predict_survey(activity, companion, pressure, tired, energetic):
    user_input = pd.DataFrame([{
        'C2': activity,
        'C3': companion,
        'C4': pressure,
        'C5': tired,
        'C6': energetic
    }])
    encoded = encoder.transform(user_input[features])
    scaled = scaler.transform(encoded)
    prediction = model.predict(scaled)[0][0]
    label = "Stressed" if prediction >= 0.5 else "Not Stressed"
    confidence = round(float(prediction if prediction >= 0.5 else 1 - prediction), 2)
    return label, confidence


AUDIO

In [7]:
import tensorflow as tf
import numpy as np
import librosa

#Audio model already loaded globally but can be loaded again.
#audio_model = tf.keras.models.load_model('/content/drive/MyDrive/saved_models/audio_model.h5')

#Function to convert the audio waveform into machine-readable 2d feature map (MFCCs)
def extract_features(audio, sr=22050):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=38)

    # Pad or crop to 98 time steps
    if mfccs.shape[1] < 98:
        pad_width = 98 - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :98]

    mfccs = np.expand_dims(mfccs, axis=-1)  # (38, 98, 1)
    return mfccs

#Function to predict stress based on the MFCC made by previous function
def predict_audio(audio_file, model=audio_model):
    try:
        y, sr = librosa.load(audio_file, sr=22050)
        features = extract_features(y, sr)
        features = features.reshape(1, 38, 98, 1)  # Input shape for CNN

        prediction = model.predict(features, verbose=0)[0][0]  # Binary classification

        label = "Stressed" if prediction >= 0.5 else "Not Stressed"
        confidence = prediction if prediction >= 0.5 else 1 - prediction

        return label, confidence

    except Exception as e:
        print(f"❌ Audio Prediction Error: {str(e)}")
        return "Error", 0.0


FUSION

In [8]:
import numpy as np

#Function to get the fused(weighted) average confidence of the prediction
def get_stress_confidence(label, confidence):
    if label.lower() == 'stressed':
        return float(confidence)
    else:
        return 1.0 - float(confidence)

def agreement_fusion(confidences):
    M = len(confidences)
    agree_scores = []
    for i in range(M):
        total_agree = 0
        for j in range(M):
            if i != j:
                total_agree += (1 - abs(confidences[i] - confidences[j]))
        agree_i = total_agree / (M - 1)
        agree_scores.append(agree_i)
    agree_scores = np.array(agree_scores)
    fused = np.sum(agree_scores * confidences) / np.sum(agree_scores)
    return fused

In [18]:
import gradio as gr

# ---------------------------------------------------------
# YOUR ORIGINAL PREDICTION LOGIC (UNCHANGED)
# ---------------------------------------------------------
def fused_stress_prediction(audio_file, photo_path,
                            activity, companion, pressure, tired, energetic):

    try:
        print("📥 Inputs Received:")
        print("Audio:", audio_file)
        print("Image:", photo_path)
        print("Survey:", activity, companion, pressure, tired, energetic)

        label_audio, conf_audio = predict_audio(audio_file, audio_model)
        label_facial, conf_facial = predict_facial(photo_path, facial_model)
        label_survey, conf_survey = predict_survey(activity, companion, pressure, tired, energetic)

        confidences = [
            get_stress_confidence(label_audio, conf_audio),
            get_stress_confidence(label_facial, conf_facial),
            get_stress_confidence(label_survey, conf_survey)
        ]

        fused_score = round(agreement_fusion(confidences), 2)
        print(f"🔁 Fused Stress Score: {fused_score:.2f}")

        label = "🧠 Stressed" if fused_score >= 0.5 else "🙂 Not Stressed"

        return f"{label} (Fused Score: {fused_score:.2f})"

    except Exception as e:
        return f"⚠️ Error: {str(e)}"


choices = [
    ["Working", "Studying", "Relaxing", "Exercising", "Other"],
    ["Alone", "Friends", "Family", "Colleagues", "Strangers"],
    ["Yes", "No", "Maybe"],
    ["Yes", "No"],
    ["Very Energetic", "Somewhat Energetic", "Neutral", "Tired"]
]


# ---------------------------------------------------------
# PREMIUM HEALTHCARE THEME COLORS
# ---------------------------------------------------------
premium_theme = gr.themes.Base(
    primary_hue="teal",
    secondary_hue="blue",
    neutral_hue="gray",
).set(
    body_background_fill="#f0f6f7",
    block_background_fill="#ffffff",
    block_title_text_color="#0d3b66",
    link_text_color="#0077b6",
    button_primary_background_fill="#1aa6b7",
    button_primary_text_color="white",
)


# ---------------------------------------------------------
# FULL WEBSITE UI
# ---------------------------------------------------------
with gr.Blocks(theme=premium_theme, css="""
    .fullpage {
        width: 90%;
        margin: auto;
        text-align: center;
        padding: 25px;
    }
    h1, h2, h3, p, li {
        text-align: center !important;
    }
    .big-section {
        max-width: 950px;
        margin: auto;
        font-size: 17.5px;
        line-height: 1.9;
    }
    .chatbot-btn {
        display: inline-block;
        background: #024059;
        padding: 12px 22px;
        border-radius: 10px;
        margin-top: 20px;
        color: white !important;
        font-weight: 600;
        font-size: 18px;
        text-decoration: none !important;
    }
    .chatbot-btn:hover {
        background: #1aa6b7;
    }
""") as demo:

    with gr.Tabs():

        # -------------------------------------------------
        # HOME PAGE (HIGHLY EXPANDED MEDICAL CONTENT)
        # -------------------------------------------------
        with gr.Tab("🏠 Home"):
            gr.HTML("""
<div class='fullpage big-section'>

<h1><b>StressAware™ – Comprehensive Multimodal Stress Assessment System</b></h1>

<p>
StressAware™ is a clinically influenced AI-driven platform designed to analyze stress through
<b>voice biomarkers</b>, <b>facial micro-expressions</b>, and <b>behavioral context</b>.
This combined approach aligns with modern research in mental-health informatics, affective science,
and digital well-being monitoring.
</p>

<hr>

<h2><b>Understanding Stress: A Medical Perspective</b></h2>

<p>
Stress is not simply an emotional experience — it is a measurable biological and neurological process.
When the human body perceives threat or pressure, the <b>Hypothalamic–Pituitary–Adrenal (HPA) axis</b> activates,
releasing cortisol, adrenaline, and inflammatory markers.
Over time, chronic activation leads to:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Impaired cognitive functioning and memory dysregulation</li>
<li>Elevated blood pressure and cardiovascular strain</li>
<li>Disrupted sleep cycles and hormonal imbalance</li>
<li>Reduced immune response and slower healing</li>
<li>Increased risk of anxiety, depression, and burnout</li>
</ul>

<p>
Modern healthcare recognizes stress as a major contributor to global disease burden.
Both <b>WHO</b> and <b>American Psychological Association (APA)</b> recommend early detection,
continuous monitoring, and scalable self-assessment tools.
</p>

<hr>

<h2><b>Why Early Stress Detection Is Crucial</b></h2>

<p>
Early identification supports better long-term health outcomes. Research shows individuals who regularly
monitor stress have:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Lower risks of burnout and chronic fatigue</li>
<li>Enhanced productivity and decision accuracy</li>
<li>Better emotional regulation and interpersonal functioning</li>
<li>Reduced dependence on crisis-stage mental health intervention</li>
<li>Greater resilience and adaptive coping skills</li>
</ul>

<hr>

<h2><b>Our Multimodal Approach to Stress Detection</b></h2>

<p>
Traditional stress evaluation often relies on subjective reporting.
StressAware™ enhances this process with clinically aligned observations across:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li><b>Voice Analysis</b> – detecting micro-tremors, vocal strain, pitch anomalies caused by stress hormones</li>
<li><b>Facial Behavior</b> – observing muscle tension, micro-expressions linked to emotional load</li>
<li><b>Contextual Indicators</b> – analyzing fatigue, perceived pressure, social environment, and energy levels</li>
<li><b>Behavior–Emotion Correlation</b> – combining multiple modalities to provide a reliable unified stress profile</li>
</ul>

<hr>

<h2><b>Evidence-Based Features of StressAware™</b></h2>

<p>
Our system integrates multidisciplinary principles from:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Affective Computing</li>
<li>Behavioral Neuroscience</li>
<li>Speech Signal Processing</li>
<li>Emotional AI & Bioinformatics</li>
<li>Digital Mental Health Frameworks</li>
</ul>

<p>
This makes StressAware™ suitable for:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Students coping with exams and academic pressure</li>
<li>Professionals dealing with occupational stress</li>
<li>Athletes tracking performance anxiety</li>
<li>Individuals engaged in wellness, therapy, or self-improvement</li>
</ul>

<p>
Proceed to the <b>Stress Detection</b> tab to receive your real-time evaluation.
</p>

</div>
            """)


        # -------------------------------------------------
        # ABOUT PAGE (EXPANDED + MEDICAL + TECHNICAL)
        # -------------------------------------------------
        with gr.Tab("📘 About the Project"):
            gr.HTML("""
<div class='fullpage big-section'>

<h1><b>About the Multimodal Stress Detection System</b></h1>

<p>
This project is built on the convergence of advanced signal processing, facial affect science,
behavioral psychology, and machine learning.
The goal is to make stress assessment accessible, data-driven, and clinically aligned.
</p>

<hr>

<h2><b>1. Voice Stress Biomarkers</b></h2>

<p>
Stress modifies the neuromuscular coordination of the vocal folds.
Research indicates that stress hormones influence:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Fundamental frequency (F0) instability</li>
<li>Amplitude modulation and loudness variability</li>
<li>Speech rate, pausing patterns, and articulation clarity</li>
<li>Micro-tremors associated with tension</li>
</ul>

<p>
Our voice model extracts these acoustic signatures and uses them to predict stress probability.
</p>

<hr>

<h2><b>2. Facial Expression & Micro-Behavior Analysis</b></h2>

<p>
Facial muscle movements are one of the most reliable windows into affective state.
Stress triggers involuntary changes such as:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Brow lifting or contraction (frontalis/corrugator activity)</li>
<li>Orbital tightening around the eyes</li>
<li>Lip pressing, corner tightening, or asymmetry</li>
<li>Identity-preserved micro-expressions lasting under 1/2 second</li>
</ul>

<p>
The system uses these micro-patterns to contribute to the multimodal stress score.
</p>

<hr>

<h2><b>3. Contextual Behavioral Indicators</b></h2>

<p>
True stress interpretation requires context.
This project captures:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li>Nature of current activity</li>
<li>Social surroundings</li>
<li>Self-reported pressure levels</li>
<li>Fatigue indicators</li>
<li>Energy levels indicating emotional–physical alignment</li>
</ul>

<p>
These factors work as behavioral co-signals to improve reliability.
</p>

<hr>

<h2><b>4. Multimodal Fusion: The Core of Clinical-Grade Analysis</b></h2>

<p>
Each modality produces an independent assessment.
The fusion mechanism then performs:
</p>

<ul style="text-align:center; list-style-position: inside;">
<li><b>Evidence aggregation</b></li>
<li><b>Confidence weighting</b></li>
<li><b>Cross-modal consistency checking</b></li>
</ul>

<p>
This leads to a scientifically aligned stress probability score, similar to
methodologies used in digital diagnostics and computational psychiatry.
</p>

<hr>

<h2><b>5. Applications and Real-World Impact</b></h2>

<ul style="text-align:center; list-style-position: inside;">
<li>Mental-health monitoring platforms</li>
<li>Employee well-being systems</li>
<li>Therapy & counseling augmentation</li>
<li>Education — student workload monitoring</li>
<li>Athletic performance optimization</li>
<li>AI-driven emotional health assistants</li>
</ul>

<p>
The project demonstrates how multimodal AI can transform digital health assessments.
</p>

</div>
            """)


        # -------------------------------------------------
        # STRESS DETECTION PAGE (unchanged except layout)
        # -------------------------------------------------
        with gr.Tab("🧪 Stress Detection"):
            gr.HTML("""
<div class='fullpage'>
<h1><b>Stress Detection Module</b></h1>
<p class='big-section'>
Upload a voice sample, facial image, and your contextual information.
Your stress score is generated in real-time using multimodal AI.
</p>
</div>
            """)

            output_box = gr.Textbox(label="🧠 Stress Assessment Result", lines=2)

            # Chatbot button OUTSIDE output area
            gr.HTML("""
<a class='chatbot-btn' href='http://127.0.0.1:8000/' target='_blank'>
💬 Open Wellness Chatbot
</a>
            """)

            def run_model(audio, img, a, c, p, t, e):
                return fused_stress_prediction(audio, img, a, c, p, t, e)

            inputs = [
                gr.Audio(type="filepath", label="🎤 Voice Input"),
                gr.Image(type="filepath", label="📸 Facial Image"),
                gr.Radio(choices=choices[0], label="Current Activity"),
                gr.Radio(choices=choices[1], label="Who are you with?"),
                gr.Radio(choices=choices[2], label="Feeling under pressure?"),
                gr.Radio(choices=choices[3], label="Feeling tired?"),
                gr.Radio(choices=choices[4], label="Current energy level")
            ]

            submit = gr.Button("Analyze Stress")
            submit.click(run_model, inputs, output_box)


# ---------------------------------------------------------
# LAUNCH
# ---------------------------------------------------------
demo.launch()


/tmp/ipython-input-838106839.py:65: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=premium_theme, css="""
/tmp/ipython-input-838106839.py:65: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=premium_theme, css="""


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26fb35dbfd354a21fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
